In [1]:
from pyspark.sql import SparkSession, HiveContext
spark = SparkSession.builder.appName('Ops').getOrCreate()

from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import DoubleType, IntegerType, StringType


In [2]:
om_out= spark.read.option("header","true").option("delimiter","|").option("inferSchema", "true").format("csv").load("/home/qx816/notebooks/weibull_mar7/OM_MODEL_OUTPUT_2019-03-12.csv")




In [3]:
om_out.printSchema()

root
 |-- CALC_ID: string (nullable = true)
 |-- CALC_DATE: timestamp (nullable = true)
 |-- CODE: string (nullable = true)
 |-- OM_CRRT_PRIORITY_SCORE: double (nullable = true)
 |-- OM_PREV_PRIORITY_SCORE: double (nullable = true)
 |-- OM_OUTPUT_01: timestamp (nullable = true)
 |-- OM_OUTPUT_02: double (nullable = true)
 |-- OM_OUTPUT_03: double (nullable = true)
 |-- SOURCE: string (nullable = true)
 |-- OM_ALGO: string (nullable = true)
 |-- OM_RUN_ANALYTICS_DATE: timestamp (nullable = true)
 |-- OM_NBR: double (nullable = true)
 |-- OM_NBR_PREV: double (nullable = true)
 |-- OM_SCORE: double (nullable = true)
 |-- OM_SCORE_DELTA: double (nullable = true)
 |-- OM_SCORE_REL: double (nullable = true)
 |-- OM_SCORE_REL_PREV: double (nullable = true)
 |-- OM_SCORE_REL_BUCKET: string (nullable = true)
 |-- OM_SCORE_REL_PREV_BUCKET: string (nullable = true)
 |-- OM_WEIBULL_SCORE_RPH_MILEAGE_MEDIAN: double (nullable = true)
 |-- OM_WEIBULL_SCORE_RPH_MONTH_IN_SERVICE_MEDIAN: double (nullabl

In [4]:
sqlContext.registerDataFrameAsTable(om_out, "om_out_df")


In [34]:
co=sqlContext.sql("select CALC_ID from om_out_df where CALC_date == (select max(calc_date) from  om_out_df) order by ")

co.show()

+--------------------+
|             CALC_ID|
+--------------------+
|1117-All-All-X15 ...|
|1144-All-All-All-EDS|
|1679-All-All-All-EDS|
|1883-All-All-ISX3...|
|1883-All-All-X15 ...|
|1921-All-All-ISX2...|
|1921-All-All-ISX3...|
|1922-All-All-ISX1...|
| 196-All-All-All-EDS|
|1963-All-All-All-...|
+--------------------+



In [27]:
co=sqlContext.sql("select OM_CRRT_PRIORITY_SCORE from om_out_df where om_algo == 'MA_Ribbon' and cast(OM_CRRT_PRIORITY_SCORE as int)<0 ")

co.count()

0

In [52]:
co=sqlContext.sql("select max(calc_date) from om_out_df")

co.show()

+--------------+
|max(calc_date)|
+--------------+
|    2019-03-07|
+--------------+



In [54]:
co=sqlContext.sql("select OM_SCORE_REL_BUCKET,count(CALC_ID) from om_out_df where calc_date == '2019-03-07' GROUP BY OM_SCORE_REL_BUCKET ")

co.show()

+-------------------+--------------+
|OM_SCORE_REL_BUCKET|count(CALC_ID)|
+-------------------+--------------+
|               High|         13525|
|                Low|          7694|
|             Middle|         11091|
|            Decline|             8|
+-------------------+--------------+

